In [1]:
import pandas as pd
import os
import json
import ollama
os.chdir('./output/3IdeaPasses/')

### Prompt to generate Utility Formula

I have a csv file with the columns: feasibility_score, co2_kg_per_year
, impact_score, and risk(either low, medium, hight). I want to provide this csv file to the LLM and prompt it to computes a weighted utility score for each row (i.e each unique value on idea_id column). Generate me such prompt.

In [4]:
#read new jason file
task4 = pd.read_json('task4.json')
task5 = pd.read_json('agent5_co2.json')
task6 = pd.read_json('agent6_impact.json')
task7 = pd.read_json('agent7_risks.json')
# merge all dfs on idea_id
whole_df = pd.merge(task4, task5, on='idea_id', how='inner')
whole_df = pd.merge(whole_df, task6, on='idea_id', how='inner')
whole_df = pd.merge(whole_df, task7, on='idea_id', how='inner')
whole_df.shape

(20, 11)

In [6]:
whole_df.head(5)

,title,description,data_needs,idea_id,feasibility_score,feasibility_notes,co2_kg_per_year,impact_score,summary,risk_level,mitigation_suggestions
0,Arctic Anomaly Detection with Computer Vision,A deep learning system that detects and classi...,"[Satellite imagery, Arctic climate data, histo...",8_arctic_anomaly_detection_with_computer_vision,0.65,Satellite imagery and climate data are partial...,1047500,0.60,should explain that the project's main impact ...,medium,[**Implement region-specific bias audits** by ...
1,Climate Resilience Prediction using GANs,Generative adversarial networks (GANs) simulat...,"[Historical climate data, urban/agricultural d...",9_climate_resilience_prediction_using_gans,0.70,Data availability is mixed with public climate...,10000,0.70,needs to explain these. The project's main ben...,high,[Implement a data diversity audit to ensure tr...
2,Sea Level Rise Prediction with Deep Learning,A deep learning model that forecasts regional ...,"[Ocean current measurements, satellite altimet...",10_sea_level_rise_prediction_with_deep_learning,0.70,Public datasets for ocean and satellite data r...,3500000,0.65,The model's predictive capabilities are essent...,medium,[**Implement geographically stratified data sa...
3,Renewable Energy Output Forecasting,Machine learning algorithms predict solar and ...,"[Weather forecasts, satellite imagery, energy ...",11_renewable_energy_output_forecasting,0.75,Public weather and satellite datasets are wide...,3000000,0.85,The project significantly reduces emissions by...,high,[**Implement geospatial bias audits** by train...
4,Climate Policy Optimization with Reinforcement...,Reinforcement learning models simulate and rec...,"[Climate policy datasets, economic indicators,...",12_climate_policy_optimization_with_reinforcem...,0.65,Data integration challenges exist due to diver...,50000,0.75,The project has a high potential to reduce emi...,high,[**Incorporate fairness-aware reward functions...


In [11]:
def query_ollama_model(model_name, PROMPT_FOR_KEYWORDS):
    """
    Sends a prompt to the specified Ollama model and returns the response.

    Args:
        model_name (str): The name of the Ollama model to use (e.g., "llama3.2:3b").
        prompt_text (str): The prompt to send to the model.

    Returns:
        str: The content of the model's response, or an error message.
    """
    try:
        print(f"Sending prompt to Ollama model: {model_name}...")
        response = ollama.chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': PROMPT_FOR_KEYWORDS,
                },
            ]
        )
        # print("Received response from Ollama.")
        return response['message']['content']
    except Exception as e:
        return f"An error occurred while communicating with Ollama: {e}"

def get_ollama_response(PROMPT_FOR_KEYWORDS,OLLAMA_MODEL_NAME="llama3.2:3b"):

    # --- Execution ---
    # print(f"Requesting keywords from Ollama for model: {OLLAMA_MODEL_NAME}...")
    
    keywords_response = query_ollama_model(OLLAMA_MODEL_NAME, PROMPT_FOR_KEYWORDS)
    return keywords_response

In [12]:
prompt1 = f"""
You are an AI Score Calculator. Your task is to compute a 'Utility Score' for the data provided.
For each query, you will receive:
*   `feasibility_score` (numerical, 0.0 to 1.0, higher is better)
*   `co2_kg_per_year` (numerical, lower is generally better)
*   `impact_score` (numerical, 0.0 to 1.0, higher is better)
*   `risk` (categorical: 'low', 'medium', or 'high')

**Calculation Instructions:**

1.  **Convert Risk to Numerical Value:**
    *   If `risk` is 'low', use `numerical_risk` = 1
    *   If `risk` is 'medium', use `numerical_risk` = 3
    *   If `risk` is 'high', use `numerical_risk` = 5
    *(A higher numerical_risk value generally detracts from the utility).*

2.  **Compute Utility Score:**
    You MUST use the following formula structure. The weights (`w_f`, `w_i`, `w_c`, `w_r`) determine how each factor contributes. **You (the user providing this prompt) need to define these weights.**

    `Utility Score = (w_f * feasibility_score) + (w_i * impact_score) - (w_c * co2_kg_per_year_scaled_or_penalized) - (w_r * numerical_risk)`

    **USER: DEFINE WEIGHTS AND CO2 HANDLING HERE:**
    *   `w_f` (Weight for feasibility): [E.G., 0.4]
    *   `w_i` (Weight for impact): [E.G., 0.4]
    *   `w_c` (Weight/factor for CO2): [E.G., 0.001]  <-- This value will directly multiply `co2_kg_per_year`. Adjust carefully based on typical CO2 values. A positive w_c means higher CO2 *reduces* the score.
    *   `w_r` (Weight for risk): [E.G., 0.2]
    *   **(Optional) CO2 Scaling/Normalization Note:** If your `co2_kg_per_year` values are very large, simply multiplying by a small `w_c` might still dominate the score or not scale well. You might instruct the model to first transform `co2_kg_per_year` (e.g., `1 / (1 + co2_kg_per_year * 0.0001)`) if you need a more normalized CO2 contribution, or adjust `w_c` very carefully. For the simplest case, `co2_kg_per_year_scaled_or_penalized` can just be `co2_kg_per_year`.

"""
prompt2 = f"""
**Task:**
Process the provided CSV data. For each row, calculate the `co2_penalty_score` and then the final `Utility Score` using the specified mappings, weights (YOU MUST FILL THESE IN THE SECTION ABOVE), and formula. Present the results clearly, showing the `idea_id` and its corresponding `Utility Score`, and ideally the intermediate `numerical_risk` and `co2_penalty_score` as well for transparency.

**Example Output Format (for one idea, adapt as needed for multiple):**

`Idea ID: idea001`
`Feasibility Score: 0.8, CO2 kg/year: 50, Impact Score: 0.9, Risk: low`
`Numerical Risk: 0.1`
`CO2 Penalty Score (benchmark [BENCHMARK_CO2_VALUE]): [calculated value]`
`Calculated Utility Score: [calculated value]`
---
"""


In [23]:
def risk_to_numerical(risk):
    """
    Convert risk level to a numerical value.
    """
    if risk == 'low':
        return 1
    elif risk == 'medium':
        return 3
    elif risk == 'high':
        return 12
    else:
        raise ValueError(f"Unknown risk level: {risk}")

  
def calculate_utility_score(
    feasibility_score: float,
    impact_score: float,
    co2_kg_per_year: float, # This will be used for co2_kg_per_year_scaled_or_penalized
    numerical_risk: int,
    w_f: float, # Weight for feasibility
    w_i: float, # Weight for impact
    w_c: float, # Weight for CO2 (applied directly to co2_kg_per_year)
    w_r: float  # Weight for risk
) -> float | None:
    """
    Computes the utility score based on the provided formula and inputs.

    Args:
        feasibility_score (float): Numerical score between 0.0 and 1.0.
        impact_score (float): Numerical score between 0.0 and 1.0.
        co2_kg_per_year (float): The CO2 emissions in kg per year.
                                 This is used as 'co2_kg_per_year_scaled_or_penalized'.
        numerical_risk (int): The numerical risk value (e.g., 1, 3, 5).
        w_f (float): Weight for feasibility_score.
        w_i (float): Weight for impact_score.
        w_c (float): Weight for co2_kg_per_year. This weight will directly
                     multiply co2_kg_per_year. A positive w_c in the formula
                     `-(w_c * co2_kg_per_year)` means higher CO2 reduces the score.
        w_r (float): Weight for numerical_risk.

    Returns:
        float | None: The calculated utility score. Returns None if numerical_risk is None.
    """
    if numerical_risk is None:
        print("Error: Cannot calculate utility score because numerical_risk is None.")
        return None

    # Using co2_kg_per_year directly as co2_kg_per_year_scaled_or_penalized
    co2_component = w_c * co2_kg_per_year

    utility_score = (w_f * feasibility_score) + \
                    (w_i * impact_score) - \
                    co2_component - \
                    (w_r * numerical_risk)
    return utility_score

In [30]:

for idx,row in whole_df.iterrows():
    idea_id = row['idea_id']
    feasibility_score = row['feasibility_score']
    co2_kg_per_year = row['co2_kg_per_year']
    impact_score = row['impact_score']
    risk = risk_to_numerical(row['risk_level'])
    
    weight_of_feasibility = 30
    weight_of_impact = 10
    weight_of_co2 = 0.00001
    weight_of_risk = 1
    # Calculate the utility score
    utility_score = calculate_utility_score(
        feasibility_score=feasibility_score,
        impact_score=impact_score,
        co2_kg_per_year=co2_kg_per_year,
        numerical_risk=risk,
        w_f=weight_of_feasibility,
        w_i=weight_of_impact,
        w_c=weight_of_co2,
        w_r=weight_of_risk
    )
    
    #add the utility score to the dataframe
    whole_df.at[idx, 'utility_score'] = utility_score
    # print(f"Utility Score for idea {idea_id}: {utility_score}")
    print(f"Utility Score for idea {idea_id}: {utility_score}")
    
# Save the updated DataFrame to a new CSV file
whole_df.to_csv('utility_scores.csv', index=False)
# Save the updated DataFrame to a new JSON file
whole_df.to_json('utility_scores.json', orient='records', lines=True)

Utility Score for idea 8_arctic_anomaly_detection_with_computer_vision: 12.024999999999999
Utility Score for idea 9_climate_resilience_prediction_using_gans: 15.900000000000002
Utility Score for idea 10_sea_level_rise_prediction_with_deep_learning: -10.499999999999996
Utility Score for idea 11_renewable_energy_output_forecasting: -11.000000000000004
Utility Score for idea 12_climate_policy_optimization_with_reinforcement_lea: 14.5
Utility Score for idea 13_sustainable_deep_learning_models_for_energy_effici: 29.633333333333326
Utility Score for idea 14_climate_data_fusion_for_enhanced_predictions: -9.166666666666664
Utility Score for idea 15_ai-driven_carbon_footprint_analysis: 13.0
Utility Score for idea 16_geoengineering_impact_assessment_with_simulations: -18.33333333333334
Utility Score for idea 17_climate_tipping_points_prediction_with_predictive_: -8.0
Utility Score for idea 18_machine_learning_for_climate_adaptation_strategies: 11.500000000000004
Utility Score for idea 19_environ